这一章讲的是ensemble方法中的VotingClassifier。这种投票的方法的主要采用多数投票或者平均预测概率来进行预测。这样可以平衡它们各自的弱点。

 Majority/Hard Voting （多数投票）
 
顾名思义，采用多数投票的规则：
- classifier 1 -> class 1
- classifier 2 -> class 1
- classifier 3 -> class 2

那么根据多数投票，将分类分成`class 1`

In [2]:
import warnings
warnings.filterwarnings("ignore")

from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.90 (+/- 0.05) [Logistic Regression]
Accuracy: 0.93 (+/- 0.05) [Random Forest]
Accuracy: 0.91 (+/- 0.04) [naive Bayes]
Accuracy: 0.95 (+/- 0.05) [Ensemble]


Weighted Majority Vote (加权的多数投票规则)

除了前一节所述的简单多数投票之外，我们还可以通过关联权重来计算加权多数投票：

$$
\hat{y} = \arg \max_i \sum^{m}_{j=1} w_j \chi_A \big(C_j(\mathbf{x})=i\big),
$$

同样的：
- classifier 1 -> class 1
- classifier 2 -> class 1
- classifier 3 -> class 2

然后我们给每个分类器加一定的权重`{0.2, 0.2, 0.6}`，那么:

$$
\arg \max_i [0.2 \times i_1 + 0.2 \times i_1 + 0.6 \times i_2] = class 2
$$

Soft Voting

Soft Voting（软投票）返回类标签作为预测概率之和的最大值。举个例子：假设一个二分类任务预测标签$i \in \{class1, class2\}$。而分类器以概率的形式返回预测结果：
- $C_1(\mathbf{x}) \rightarrow [0.9, 0.1]$
- $C_2(\mathbf{x}) \rightarrow [0.8, 0.2]$
- $C_3(\mathbf{x}) \rightarrow [0.4, 0.6]$

计算平均的概率：

$$
p(i_0 \mid \mathbf{x}) = \frac{0.9 + 0.8 + 0.4}{3} = 0.7 \\\\
p(i_1 \mid \mathbf{x}) = \frac{0.1 + 0.2 + 0.6}{3} = 0.3
$$

$$
\hat{y} = \arg \max_i \big[p(i_0 \mid \mathbf{x}), p(i_1 \mid \mathbf{x}) \big] = class1
$$

In [10]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier

# Loading some example data
iris = datasets.load_iris()
X = iris.data[:, [0,2]]
y = iris.target

# Training classifiers
clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)], voting='soft', weights=[2,1,2])

PRML中的理论知识
考虑⼀个回归问题，其中我们试图预测⼀个连续变量的值，并且假设我们⽣成了$M$个⾃助数据集然后使⽤每个数据集训练处了预测模型的⼀个独⽴的副本$y_m(x)$

$$
y_{COM}(x) = \frac{1}{M}\sum\limits_{m=1}^My_m(x) \tag{14.7}
$$

而$y_m(x)$可以做以下的表示：

$$
y_m(x) = h(x) + \epsilon_m(x) \tag{14.8}
$$
![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH14/bagging.jpg)

### 各个模型独⽴预测的平均误差

$$
\mathbb{E}_x[{y_m(x) - h(x)}^2] = \mathbb{E}_x[\epsilon_m(x)^2] \tag{14.9}
$$

$$
E_{AV} = \frac{1}{M}\sum\limits_{m=1}^M\mathbb{E}_x[\epsilon_m(x)^2] \tag{14.10}
$$

### committees的期望误差

![](https://raw.githubusercontent.com/data2world/PRML_Note/master/IMG/CH14/14-11.png)



如果我们假设误差的均值为零，且不具有相关性，即:
$$
\begin{eqnarray}
\mathbb{E}_x[\epsilon_m(x)] &=& 0 \tag{14.12} \\
\mathbb{E}_x[\epsilon_m(x)\epsilon_l(x)] &=& 0 , m \neq l \tag{14.13}
\end{eqnarray}
$$

那么我们有:
$$
E_{COM} = \frac{1}{M}E_{AV} \tag{14.14}
$$

上式说明：一个模型的平均误差可以仅仅通过对模型的$M$个版本求平均的方式减小$M$倍。不幸的是，它依赖于我们的关键假设，即由各个单独的模型产生的误差是不相关的。在实际应用中，误差通常是高度相关的，因此整体的误差下降是通常是很小的。